In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from os import path

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs detected!")

1 Physical GPUs, 1 Logical GPUs


In [4]:
image_generator = ImageDataGenerator()

In [5]:
input_dir = "lettersDataset_final"

In [6]:
size = (50, 50)

In [7]:
train_generator = image_generator.flow_from_directory(path.join(input_dir, "train"), target_size=size, class_mode="sparse")

Found 243282 images belonging to 36 classes.


In [24]:
test_generator = image_generator.flow_from_directory(path.join(input_dir, "test"), target_size=size, class_mode="sparse", shuffle=False)

Found 60806 images belonging to 36 classes.


In [8]:
from model import make_model
w,h = size
model = make_model((w, h, 3))

In [7]:
checkpoint_path="checkpoints\\chk1"

In [28]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)

In [29]:
history = model.fit(train_generator, validation_data=test_generator, callbacks=[checkpoint_callback], epochs=5)

Epoch 1/5
7603/7603 [==============================] - 247s 33ms/step - loss: 0.5869 - accuracy: 0.8959 - val_loss: 0.4247 - val_accuracy: 0.9255
Epoch 2/5
7603/7603 [==============================] - 160s 21ms/step - loss: 0.3817 - accuracy: 0.9326 - val_loss: 0.4017 - val_accuracy: 0.9302
Epoch 3/5
7603/7603 [==============================] - 159s 21ms/step - loss: 0.3097 - accuracy: 0.9423 - val_loss: 0.4010 - val_accuracy: 0.9316
Epoch 4/5
7603/7603 [==============================] - 187s 25ms/step - loss: 0.2350 - accuracy: 0.9519 - val_loss: 0.4767 - val_accuracy: 0.9288
Epoch 5/5
7603/7603 [==============================] - 554s 73ms/step - loss: 0.1695 - accuracy: 0.9611 - val_loss: 0.5687 - val_accuracy: 0.9202


In [30]:
model.evaluate(test_generator)

1901/1901 [==============================] - 22s 11ms/step - loss: 0.5687 - accuracy: 0.9202


[0.5686889290809631, 0.9202216863632202]

In [33]:
latest = tf.train.latest_checkpoint(checkpoint_path)

In [34]:
model.load_weights(latest)

In [35]:
model.evaluate(test_generator)

1901/1901 [==============================] - 20s 11ms/step - loss: 0.4010 - accuracy: 0.9316


[0.4010215401649475, 0.9316350221633911]

In [36]:
model.save(checkpoint_path)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: captcha\assets
